<a href="https://colab.research.google.com/github/B-Mur/forensic-data-explorer/blob/master/ImageViewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>ImageViewer</h1>
This notebook is setup so that each image pair from the handheld FLIR sensor can be viewed in conjunction with the ambient temperature and logging data (if provided on a subject. 

This notebook has been setup so that the minimal amount of code needs to be directly interacted with. That being said, this notebook is missing some functionality, but should be able to do basics in it. 



The first thing we need to do is import all the necessary packages.  

In [ ]:
# if it's the first time running 
!git clone https://github.com/B-Mur/nij-data-explorer.git
!pip install flirimageextractor
!pip install loguru
!apt-get install -y exiftool
import matplotlib as mlp
import numpy as np
import datetime
import pandas as pd
import flirimageextractor
import os
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from PIL import Image
import sys
sys.path.append('/content/nij-data-explorer')
from utils import *



The next block you will need to fill in your google drive folder path in the `imgDir` variable. This will then plot all of the dates and times of the images taken. 

In [ ]:
imgDir = '/content/drive/My Drive/NIJ/Notebooks/Data/NIJ-DATA/DATA/2020_017'
df = build_duo_image_links(imgDir)
df['Time'], df['Date'] = df['DateTime'].dt.time, df['DateTime'].dt.date
df.sort_values(by='Time', inplace=True)
px.scatter(x=df['Date'], y=df['Time'])

In the following, block of code, it will plot all rgb and thermal image pairs for the date specified in `DATE`. 

Be sure to follow the following format -> `XXX YY ZZZZ` where `XXX` is the first three letters of the month, `YY` is the date, and `ZZZZ` is the year. So, `Mar 24 2020` is a valid date. 

In [ ]:
# Change this date
DATE = 'May 11 2020'


# Don't touch this -> It's basically converting to the right format. 
DATE = datetime.datetime.strptime(DATE, '%b %d %Y')
DAY_DF = df.loc[pd.to_datetime(df.Date)==DATE]

weather = pd.read_csv('/content/drive/My Drive/NIJ/Notebooks/Data/NIJ-DATA/WEATHER/030120_071620.csv', header=6)

weather['Date_Time'] = pd.to_datetime(weather['Date_Time'])
dt = DAY_DF.DateTime.iloc[ind]
current_weather = weather.loc[weather.Date_Time.dt.date == dt]
logger_fig = go.Figure()
logger_fig.add_trace(go.Scatter(x=current_weather['Date_Time'], y=current_weather['Air Temp'], name='Air Temperature'))
logger_fig.add_trace(go.Scatter(x=current_weather['Date_Time'], y=current_weather['relative_humidity_set_1'], name='Relative Humidity'))
logger_fig.add_trace(go.Scatter(x=current_weather['Date_Time'], y=current_weather['soil_moisture_set_1'], name='Soil Moisture'))
logger_fig.add_trace(go.Scatter(x=current_weather['Date_Time'], y=current_weather['solar_radiation_set_1'], name='Soil Radiation'))
logger_fig.add_trace(go.Scatter(x=current_weather['Date_Time'], y=current_weather['soil temp '], name='Soil Temperature'))

logger_fig.show()



for ind in range(0, DAY_DF.shape[0], 2):
  imgs = DAY_DF.loc[DAY_DF.DateTime==DAY_DF.DateTime.iloc[ind]]
  imgs.reset_index(inplace=True, drop=True) 
  print(DAY_DF.DateTime.iloc[ind])
  

  for i, row in imgs.iterrows():
    imgPath = os.path.join(imgDir, imgs.loc[i][0])
    img, modality = get_flir_image(imgPath)
    if modality == 'thermal':
      thermal = np.flip(img, axis=0)
    elif modality == 'rgb':
      rgb = img

  img_fig = plot_rgb_thermal(thermal, rgb, dynamic = [10, 60])
# TODO Insert Weather Histogram



  img_fig.show()

  

